In [1]:
import json
import numpy as np
import pandas as pd
import math
import ast

In [44]:
# with open('dataset/business_LV.json') as fd:
#     df = json.load(fd)
# print(len(df))
df = pd.read_csv('dataset/business_LV.csv')
print(len(df))

3923


In [45]:
# ["business_id", "neighborhood", "stars", "review_count", 
#             "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups", 
#             "RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]

In [46]:
def variables(all_columns,mode):
    cols = all_columns
    cat_cols = ["neighborhood", "RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking", "categories"]
    if mode == "cat":
        cols = ["neighborhood", "categories"]
    elif mode == "bool":
        cols = ["RestaurantsTakeOut", "GoodForKids", "RestaurantsGoodForGroups","RestaurantsReservations","DriveThru", "WiFi","BusinessParking"]
    elif mode == "cont":
        cols = ['stars', 'review_count']
    print(cols)
    return cols

In [48]:
# def format_long_data_cat(df,cols_to_feat,time):
    #First 6 hours
cols_to_feat = []
new_df = df.copy()
new_df = new_df.sort_values(['business_id'])
id_list = sorted(list(set(list(new_df['business_id']))))
cols = variables(new_df.columns,"cat")

for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        for col in cols:
            if (col == 'categories'):
                options = ast.literal_eval(row[col])
                for option in options:
                    if (option != "Restaurants" and option not in cols_to_feat):
                        cols_to_feat.append(option)
            else:
                suffix = row[col]
                if (type(suffix) is float and math.isnan(suffix)):
                    suffix = "missing"
                else:
                    suffix = row[col]
                if (col+'_'+suffix) not in cols_to_feat:
                    cols_to_feat.append(col+'_'+suffix)
print(cols_to_feat)


['neighborhood', 'categories']
['neighborhood_The Strip', 'Steakhouses', 'Cajun/Creole', 'Seafood', 'Bars', 'Nightlife', 'American (New)', 'neighborhood_missing', 'Thai', 'neighborhood_Eastside', 'Buffets', 'neighborhood_University', 'Burgers', 'Korean', 'Fast Food', 'neighborhood_Spring Valley', 'Comfort Food', 'Specialty Food', 'Asian Fusion', 'Food', 'neighborhood_Westside', 'Dive Bars', 'Indian', 'Bar Crawl', 'American (Traditional)', 'Chinese', 'Mexican', 'neighborhood_South Summerlin', 'Japanese', 'neighborhood_Sunrise', 'Latin American', 'Salvadoran', 'neighborhood_Chinatown', 'Noodles', 'Vietnamese', 'neighborhood_Southeast', 'Hawaiian', 'South African', 'African', 'Ramen', 'neighborhood_Summerlin', 'Middle Eastern', 'Mediterranean', 'Lebanese', 'Moroccan', 'Breakfast & Brunch', 'Event Planning & Services', 'Lounges', 'Venues & Event Spaces', 'Casinos', 'Arts & Entertainment', 'Beer', 'Wine & Spirits', 'Music Venues', 'Pubs', 'Beer Bar', 'Diners', 'Sushi Bars', 'neighborhood_So

In [49]:
cat_arr = np.zeros((len(id_list),len(cols_to_feat)))
print(cat_arr.shape)
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros((len(cols_to_feat)))
        j = 0
#         print(row)
        for col in cols:
            if (col == "categories"):
                options = ast.literal_eval(row[col])
                for option in options:
                    if (option != "Restaurants"):
                        feature_vector[cols_to_feat.index(option)] = 1
            else:
                suffix = row[col]
                if (type(suffix) is float and math.isnan(suffix)):
                    suffix = "missing"
                else:
                    suffix = row[col]
                feature_vector[cols_to_feat.index(col+'_'+suffix)] = 1
        cat_arr[i] = feature_vector
        i+=1
print(len(cols_to_feat))

(3923, 374)
374


In [50]:
cols = variables(new_df.columns,"bool")
for col in cols:
    cols_to_feat.append(col)
bool_arr = np.zeros((len(id_list), len(cols)))
i = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros((len(cols)))
        j = 0
        for col in cols:
            if bool(row[col]):
                feature_vector[cols.index(col)] = 1
        bool_arr[i] = feature_vector
        i+=1
print(bool_arr.shape)
print(len(cols_to_feat))

['RestaurantsTakeOut', 'GoodForKids', 'RestaurantsGoodForGroups', 'RestaurantsReservations', 'DriveThru', 'WiFi', 'BusinessParking']
(3923, 7)
381


In [53]:
# cols= variables(new_df.columns, "cont")
li1 = []
c = "review_count"
# for c in cols:
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        li1.append(row[c])


In [54]:
print(len(cols_to_feat))
count_arr=np.zeros((len(id_list), 5))
temp = pd.Series(li1)
bins = list(pd.qcut(pd.Series(li1),5,duplicates='drop',retbins=True)[1])
print(bins)
for i in range(len(bins)-1):
    cols_to_feat.append(c+str(bins[i])+'_'+str(bins[i+1]))
j = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        review_count = row[c]
#         print(review_count)
        feature_vector = np.zeros(5)
        for i in range(len(bins)-1):
            if bins[i] <= review_count and bins[i+1] >= review_count:
                feature_vector[i] = 1
                break
        count_arr[j] = feature_vector
        j+=1
print(count_arr[0])
print(len(cols_to_feat))

381
[3.0, 14.0, 38.0, 95.0, 261.0, 6979.0]
[ 0.  0.  0.  0.  1.]
386


In [55]:
rating_arr = np.zeros((len(id_list), 1))
c = "stars"
cols_to_feat.append(c)
j = 0
for id in id_list:
    temp = new_df[new_df['business_id']==id]
    for idx,row in temp.iterrows():
        feature_vector = np.zeros(1)
        feature_vector[0] = float(row[c])/5.0
        rating_arr[j] = feature_vector
        j+=1
print(rating_arr[0])

[ 0.8]


In [56]:
complete_data = np.concatenate((cat_arr,bool_arr,count_arr,rating_arr),axis=1)
print(complete_data.shape)
print(len(cols_to_feat))


(3923, 387)
387


In [62]:
def getIDPath():
    return 'dataset/ids.npy'

def getFeaturesPath():
    return 'dataset/features.npy'

def getDictPath():
    return 'dataset/cols2Dict.npy'

In [63]:
np.save(getFeaturesPath(),complete_data)
np.save(getDictPath(),cols_to_feat)
np.save(getIDPath(),id_list)